# 3. Tool to adjust the product and maintenance events in CREAM

<p> This jupyter notebook was used to adjust the timestamps of maintenance and product events. The timestamps from the events were generated with a minute precision. For most applications more detailed timestamps are necessary, hence we adapted them by hand to allow for extensive evaluation of applications like condition monitoring. 
    <b> This is the third, and last, notebook in the labeling pipeline of CREAM. </b> </p>
<div class="alert alert-info">
    <h3>Instructions for using the adjustment notebook</h3>
    <p> First do all the product events, then the maintenance events for a particular day. </p>
    <p> For each event, the event itself and a +/- 1 minute window of data is plotted. </p>
    <p> The event was generated by the coffee maker after the procedure was finshed, hence, it can be located in the 60 seconds before or after the recorded event. </p>
    <p> Label the event according to the closest match. </p>
    <p> For each event, denote the start and end of the event by clicking two times into the graphic. </p>
    <p> Then you can load the next product / maintenance event. </p>
    <p> After you are done with a particular day, change the CURRENT_DAY variable accordingly, and reexecute all cells (can be found in the toolbar: Cell --> Run All).  </p>
    <p> Then proceed with the labeling. </p>
    <p> In case of interruptions or errors, you can proceed the labeling process at an arbitrary event index.</p>
    <p> Just specify the respective parameter in the display_initial_event() method in the labeling cell at the bottom of the notebook.</p> 
    <p> When doing so, be aware of the fact, that the events recorded that are still in memory are NOT deleted when continuing at an arbitrary index.</p> 
    <p> Reexecute the whole notebook if you want to reset everything and restart at an arbitrary index, after settting the corresponding parameter.</p>

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import h5py
import pandas as pd
import os
import sys
from pathlib import Path
from datetime import datetime
from datetime import timedelta
import math
import pdb
import scipy
# Add project path to path for import
project_path = os.path.abspath("..")
if project_path not in sys.path:
    sys.path.append(project_path)

# Add module path to path for import
module_path = os.path.abspath("../data_utility/data_utility.py")
if module_path not in sys.path:
    sys.path.append(module_path)
    
from data_utility import CREAM_Day # class to work with a day of the CREAM Dataset

%matplotlib notebook
# Intentional replication is necessary
%matplotlib notebook
%load_ext autoreload
# Reload all modules every time before executing the Python code typed.
%autoreload 2 
# Import some graphical modules
from IPython.display import display, clear_output
from ipywidgets import Button, Layout, ButtonStyle, HBox, VBox, widgets, Output
from IPython.display import SVG, display, clear_output

import subprocess
import glob

## Global Functions

In [ ]:
def plot_event_window(event_timestamp:pd.Timestamp, event_type:str, window_size:int, current_CREAM_day:CREAM_Day, concurrent_events_dict:dict):
    """
    
    Plots a window of window_size in each direction around the event_timestamp.
    The event_timestamp marks the beginning of the minute where the event stopped.
    So instead of directly using the event_timestamp, we plot the event_timestamp + 59 seconds 
    to mark the end of the minute in that the event stopped.
    Therefore the event has happended before the point that is marked as a bold red line.
    The current signal of the coffee maker is plotted.
    The event type is the label the event gets.
    If a concurrent_events_dict is provided, with the keys being the name of the event list and the values being the event dataframes,
    all other events that happen within the window of interest are also plotted.
    Appliance events are bold orange lines.
    Other events are dashed red lines.

    """
    
    # Import and set globals necessary for the click functions
    
    global EVENT_TYPE
    global EVENT_TIMESTAMP
    global WINDOW_START_TS
    global EVENT_DICTIONARY
    
    # Instead of taking the event timestamp directly we take the END of the minute
    end_event_timestamp  = event_timestamp + timedelta(seconds=59)
    # Tackle border cases of the timestamp
    if end_event_timestamp - timedelta(seconds=window_size) < current_CREAM_day.minimum_request_timestamp: # in case we are at the beginning of the day
        duration_to_left = end_event_timestamp - current_CREAM_day.minimum_request_timestamp
        duration_to_left = duration_to_left.total_seconds() # amount of data that we load now to the left of the current timestmap
        duration_to_right = window_size #to the right we can load the full window 
        
    elif end_event_timestamp + timedelta(seconds=window_size) >  current_CREAM_day.maximum_request_timestamp: # in case we are at the end of the day
        duration_to_right = current_CREAM_day.maximum_request_timestamp - end_event_timestamp 
        duration_to_right = duration_to_right.total_seconds() #amount of data that we load now to the right of the current timestamp
        duration_to_left = window_size #to the left we can load the full window
    
    else: # if we have anough datapoints to the left and to the right to load the full WINDOW_SIZE in each direction
        duration_to_left = window_size
        duration_to_right = window_size
    
    # Create the start- and end-timestamp and compute the overall duration of the window
    duration = duration_to_left + duration_to_right
    
    start_ts = end_event_timestamp - timedelta(seconds=duration_to_left)
        
    end_ts = end_event_timestamp + timedelta(seconds=duration_to_right)
        
    # Load the data
    
    voltage, current = current_CREAM_day.load_time_frame(start_datetime=start_ts, duration=duration) #and WINDOW_SIZE seconds after the event
        
    # Compute the index of the event, using the timestamp
    end_event_index = current_CREAM_day.get_index_from_timestamp(start_ts, end_event_timestamp)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current))
    ax.plot(xticks, current, markersize=0.1, alpha=0.6) 
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels

    if np.max(current) < 1: #in case of noise, show an appropriate range
        ax.set_ylim([-6,6])
    
    # Plot the event line
    ax.axvline(end_event_index, color="red", linewidth=1.5, label=event_type)
    
    # Add other events that happend within the window
    if len(concurrent_events_dict) > 0:
        
        for event_list_name, concurrent_events_df in concurrent_events_dict.items():
            
            # If an already refined timestamp list (either product, or maintenance) is provided, one
            # can plot the detailed end timestamps instead of the coarse grained ones that are not refined yet
            if "End_Timestamp" in concurrent_events_df.columns:
                ts_column_name = "End_Timestamp"
            else:
                ts_column_name = "Timestamp"
                
            concurrent_events_df_roi = concurrent_events_df[(concurrent_events_df[ts_column_name] <= end_ts) & (concurrent_events_df[ts_column_name] >= start_ts)]
            
            if len(concurrent_events_df_roi) > 0:
                for i, row in concurrent_events_df_roi.iterrows():
                    
                    # Get the event index
                    i = current_CREAM_day.get_index_from_timestamp(start_ts, row[ts_column_name])

                    # Some plotting adjustments, depending on the type of event that is plotted
                    if "appliance" in event_list_name:
                        color ="orange"
                        
                        if row.Component != "unlabeled":
                            label = row.Component
                            linewidth=1.5
                            
                            # In case we have a already labeled component, we add also add ht label text
                            plt.text(i, 0.5, label[0])
                            
                        else:
                            label = ""
                            linewidth=0.6
                            
                    else: # in case of product or maintenance events
                        color="red"
                        if "product" in event_list_name:
                            if "Product" in concurrent_events_df_roi.columns:
                                label = row.Product
                            elif "Event_Type" in concurrent_events_df_roi.columns:
                                label= row.Event_Type
                            else:
                                label = "unspecified"
                            
                            linewidth=1.2
                                
                        elif "maintenance" in event_list_name:
                            
                            if "Activity" in concurrent_events_df_roi.columns:
                                label = row.Activity
                            elif "Event_Type" in concurrent_events_df_roi.columns:
                                label= row.Event_Type
                            else:
                                label = "unspecified"
                            linewidth=1.2
                        else:
                            label = "Unknown"
                            linewidth=0.6
                    
                    # Plot the line
                    ax.axvline(i, color=color, linestyle=":", label=label, linewidth=linewidth)
                    
    # Display if there are any already labeled events, starting after the first window
    
    if len(EVENT_DICTIONARY["Start_Timestamp"]) > 1:
        concurrent_events_df = pd.DataFrame(EVENT_DICTIONARY)

        # use mask here because of misaligned indices 
        mask = (concurrent_events_df.End_Timestamp <= end_ts) & (concurrent_events_df.End_Timestamp >= start_ts)
        concurrent_events_df_roi = concurrent_events_df.loc[mask.values]
        
        for i, row in concurrent_events_df_roi.iterrows():
            i = current_CREAM_day.get_index_from_timestamp(start_ts, row.End_Timestamp)
            ax.axvline(i, color="green", linestyle=":", label="already labeled end " + str(i))
    
    
    # add time information to plot
   
    samples_per_minute = current_CREAM_day.sampling_rate * 60 #every 60 seconds
    
    if len(current) % samples_per_minute  == 0: #just in case the parameters are changed and there are no full minutes in the signals
        step = len(current) / samples_per_minute
        for i in range(0, int(step+1)):
            ax.axvline(i*samples_per_minute, color="black", ymax=0.1)
    
    fig.suptitle("Event type: " + event_type + "\n" + str(str(start_ts) + " - " + str(end_ts)))
    ax.legend(loc='upper right')
    
    EVENT_TYPE = event_type
    EVENT_TIMESTAMP = event_timestamp
    WINDOW_START_TS = start_ts
    
    return fig, ax

##  Global Variables

In [ ]:
EVENT_INDEX = 0
EVENT_TYPE = None
EVENT_TIMESTAMP = None
WINDOW_START_TS = None
EVENT_DICTIONARY = {
    "Start_Timestamp": [],
    "Automatic_Timestamp" : [],
    "Event_Type" : [],
    "End_Timestamp" : [],
    "Event_Duration_Seconds": []
}

LABELED_TIMESTAMP = (None, None) #the last labeled start, end timestamp

WINDOW_SIZE = 120 #seconds, the window size in each direction around and event to be displayed

TIMES_PER_PRODUCT_ACTIVITY = {'cappuccino' : 75, 
                     'coffee' : 63,
                     'espresso' : 48, 
                     'espresso_macchiato' : 110,
                     'hot_water' : 50, 
                     'latte_macchiato' : 110, 
                     'ristretto' : 33,
                     'white_coffee' : 95,
                     'Clean' : 20 * 60,
                      'CleanMilkSystem' : None,
                      'MillingPlantCoffee' : None, 
                      'MillingPlantEspresso' : None, 
                      'RinseCoffeeSystem' : None, 
                      'RinseMilkSystem' : None,
                      'Time2Clean' : None, 
                      'Time2Descale' : 50 * 60}


## Widget functions for the UI

In [ ]:
def onclick(event):
    """
    Function to be executed in case of a click event at a figure.
    """
    global LABELED_TIMESTAMP # the tuple of the timestamps labeled (start, end)
    global EVENT_TYPE #type of the event of interest
    global EVENT_TIMESTAMP #timestamp of the event of interest that was autoamticcaly generated
    global WINDOW_START_TS #start timestamp of the window we are currently looking at
    global current_CREAM_day #object representing the current day in the CREAM dataset
    global FIG #global figure object
    global AX #global axis object
    global EVENT_DICTIONARY
    
    
    if current_CREAM_day.get_timestamp_from_index(WINDOW_START_TS, math.floor(event.xdata)) > EVENT_TIMESTAMP + timedelta(seconds=60):
        print("The red timestamp is generated after the event is completed! Hence, do not place the click after it!")
        return
    
    if LABELED_TIMESTAMP[0] is None:
        
        LABELED_TIMESTAMP = (math.floor(event.xdata), None)
        AX.axvline(LABELED_TIMESTAMP[0], color="yellow", linewidth=1.5)
        FIG.show()
    elif LABELED_TIMESTAMP[0] is not None and LABELED_TIMESTAMP[1] is None :
        
        if LABELED_TIMESTAMP[0] > event.xdata: #if the second click is before the first one
            print("Please place the second click after the first one!")
            return
    
        
        LABELED_TIMESTAMP = (LABELED_TIMESTAMP[0], math.floor(event.xdata))
        AX.axvline(LABELED_TIMESTAMP[1], color="yellow", linewidth=1.5)
        
        FIG.show()
    else:
        print("Already selected two borders")
        return

In [ ]:
def on_remove_last_click(event):
    """
    Delete the last click
    """
    global FIG #global figure object
    global AX #global axis object
    global LABELED_TIMESTAMP # the tuple of the timestamps labeled (start, end)
    if LABELED_TIMESTAMP == (None, None):
        print("No click to be deleted!")
        return
    
    if LABELED_TIMESTAMP[1] is None: #if the second is None, then the first was the last click
        LABELED_TIMESTAMP = (None, None)
    
    elif None not in LABELED_TIMESTAMP: #both are not None, the the second gets deleeated
        LABELED_TIMESTAMP = (LABELED_TIMESTAMP[0], None)
        
    AX.lines[-1].remove()
    
    FIG.show()

In [ ]:
def display_initial_event(event_index_p=0):
    """
    Display the start event. This is set to 0 as per default!
    In case of interruptions in the labeling process or in case of errors, you can restart labeling at
    an arbitrary index using the event_index_p paramter.
    """
    
    global EVENT_LABELING_MODE #either product mode or maintenance mode
        
    global EVENT_INDEX # index of the last event loaded
    global EVENT_TYPE
    global maintenance_events #dataframe containing all the maintenance events
    global component_events #dataframe containing all the appliance events
    global current_CREAM_day #object representing the current day in the CREAM dataset
    global WINDOW_SIZE #window size used to display the data
    global LABELED_TIMESTAMP # the tuple of the timestamps labeled (start, end)
    global FIG #global figure object
    global AX #global axis object    
  
    
    EVENT_INDEX = event_index_p
    
    plt.clf()
    clear_output()
    
    
     # Depending on the Mode, either use the product or maintenance events
    if "product" in EVENT_LABELING_MODE:
        
        if EVENT_INDEX > len(product_events)-1:
            print("THIS WAS THE LAST EVENT! YOU ARE DONE!")
            return 
        product_row = product_events.iloc[EVENT_INDEX]
        number_of_events = len(product_events)
        print("This is event number " + str(EVENT_INDEX) + " of " + str(number_of_events))
        
        event_type = product_row.Product
        FIG, AX = plot_event_window(event_timestamp = product_row.Timestamp, 
            event_type = product_row.Product,
            window_size = WINDOW_SIZE,
            current_CREAM_day = current_CREAM_day,
            concurrent_events_dict = {
              "maintenance_events" : maintenance_events,
              "component_events" : component_events
            })
        
    elif "maintenance" in EVENT_LABELING_MODE:
        if EVENT_INDEX > len(maintenance_events)-1:
            print("THIS WAS THE LAST EVENT! YOU ARE DONE!")
            return 
        
        maintenance_row = maintenance_events.iloc[EVENT_INDEX]
        number_of_events = len(maintenance_events)
        print("This is event number " + str(EVENT_INDEX) + " of " + str(number_of_events))
        
        event_type = maintenance_row.Activity
        FIG, AX = plot_event_window(event_timestamp = maintenance_row.Timestamp, 
            event_type = maintenance_row.Activity,
            window_size = WINDOW_SIZE,
            current_CREAM_day = current_CREAM_day,
            concurrent_events_dict = {
              "product_events" : product_events,
              "component_events" : component_events
            })
        
    else:
        raise ValueError("This event labeling mode is not available. Please select maintenance or product!")
    
    if TIMES_PER_PRODUCT_ACTIVITY[event_type] is not None:
        print(str(event_type) + " " + str(TIMES_PER_PRODUCT_ACTIVITY[event_type]) + " seconds " + " or " + str(TIMES_PER_PRODUCT_ACTIVITY[event_type] * current_CREAM_day.sampling_rate))
    
    FIG.show()
    display(button_box)
    LABELED_TIMESTAMP = (None, None)

In [ ]:
def on_next_clicked(event):
    
    global EVENT_LABELING_MODE #either product mode or maintenance mode
    
    global EVENT_INDEX # index of the last event loaded
    global maintenance_events #dataframe containing all the maintenance events
    global component_events #dataframe containing all the appliance events
    global current_CREAM_day #object representing the current day in the CREAM dataset
    global WINDOW_SIZE #window size used to display the data
    global LABELED_TIMESTAMP # the tuple of the timestamps labeled (start, end)
    global FIG #global figure object
    global AX #global axis objexct
    global OUTPUT #global output widget
    global LABEL_DESTINATION_PATH #location where the event labels will be stored, is user specified
    global EVENT_DICTIONARY #global event dictionary
    global EVENT_TYPE #type of the event of interest
    global EVENT_TIMESTAMP #timestamp of the event of interest that was autoamticcaly generated
    global WINDOW_START_TS #start timestamp of the window we are currently looking at
    global current_CREAM_day #object representing the current day in the CREAM dataset
    
    if None in LABELED_TIMESTAMP:
        print("Click another Time")
        return 
    
    # first add the last event to the labels and save it
    
    start_ts = current_CREAM_day.get_timestamp_from_index(WINDOW_START_TS, LABELED_TIMESTAMP[0] )
    end_ts = current_CREAM_day.get_timestamp_from_index(WINDOW_START_TS, LABELED_TIMESTAMP[1] )
    duration_sec = end_ts - start_ts
    duration_sec = duration_sec.total_seconds()
    
    EVENT_DICTIONARY["Start_Timestamp"].append(start_ts)
    EVENT_DICTIONARY["Event_Type"].append(EVENT_TYPE)
    EVENT_DICTIONARY["Automatic_Timestamp"].append(EVENT_TIMESTAMP)
    EVENT_DICTIONARY["End_Timestamp"].append(end_ts)
    EVENT_DICTIONARY["Event_Duration_Seconds"].append(duration_sec)
            
    EVENT_INDEX += 1
    if EVENT_INDEX <= 0: #Must not be negative 
        print("This is the first event, you can not go further back in time!")
        return 
    
    
    plt.clf()
    clear_output()
    
    
    # Depending on the Mode, either use the product or maintenance events
    if "product" in EVENT_LABELING_MODE:
        
        if EVENT_INDEX > len(product_events)-1:
            print("THIS WAS THE LAST EVENT! YOU ARE DONE!")
            return 
        product_row = product_events.iloc[EVENT_INDEX]
        number_of_events = len(product_events)
        print("This is event number " + str(EVENT_INDEX) + " of " + str(number_of_events))
        
        event_type = product_row.Product

        FIG, AX = plot_event_window(event_timestamp = product_row.Timestamp, 
            event_type = product_row.Product,
            window_size = WINDOW_SIZE,
            current_CREAM_day = current_CREAM_day,
            concurrent_events_dict = {
              "maintenance_events" : maintenance_events,
              "component_events" : component_events
            })
        
    elif "maintenance" in EVENT_LABELING_MODE:
        if EVENT_INDEX > len(maintenance_events)-1:
            print("THIS WAS THE LAST EVENT! YOU ARE DONE!")
            return 
        
        maintenance_row = maintenance_events.iloc[EVENT_INDEX]
        number_of_events = len(maintenance_events)
        print("This is event number " + str(EVENT_INDEX) + " of " + str(number_of_events))
        
        event_type = maintenance_row.Activity
        FIG, AX = plot_event_window(event_timestamp = maintenance_row.Timestamp, 
            event_type = maintenance_row.Activity,
            window_size = WINDOW_SIZE,
            current_CREAM_day = current_CREAM_day,
            concurrent_events_dict = {
              "product_events" : product_events,
              "component_events" : component_events
            })
        
    else:
        raise ValueError("This event labeling mode is not available. Please select maintenance or product!")
    
    if TIMES_PER_PRODUCT_ACTIVITY[event_type] is not None:
        print(str(event_type) + " " + str(TIMES_PER_PRODUCT_ACTIVITY[event_type]) + " seconds " + " or " + str(TIMES_PER_PRODUCT_ACTIVITY[event_type] * current_CREAM_day.sampling_rate))
    
    FIG.show()
    display(button_box)
    LABELED_TIMESTAMP = (None, None)
    
    # save the last labeled event
    save_labels(destination=LABEL_DESTINATION_PATH)

In [ ]:
def save_labels(destination: str):
    global EVENT_DICTIONARY
    global EVENT_INDEX
    global EVENT_LABELING_MODE
    global CURRENT_DAY
    
    events_df = pd.DataFrame(EVENT_DICTIONARY)
    filename = "labels" + "_" + EVENT_LABELING_MODE + "_" + str(CURRENT_DAY) + ".csv" 
    
    if EVENT_INDEX % 10 == 0 and EVENT_INDEX > 0: #every 10 events: before storing the new file, save the old one       
        os.rename(os.path.join(destination, filename), os.path.join(destination, "previous_labels.csv"))
    
    #Store the new one
    events_df.to_csv(os.path.join(destination, filename), index=False)

In [ ]:
def on_delete_clicked(event):
    """
    Deletes the last click from every key in the event_dictionary and returns to the previous window
    """
    global EVENT_DICTIONARY
    global EVENT_INDEX
    global LABELED_TIMESTAMP
    global FIG
    global AX
    
    if EVENT_INDEX <= 0: #we arrived at the first event again
        print("This is the first event, you can not go further back in time!")
        return 
    
    for k, v in EVENT_DICTIONARY.items():
        EVENT_DICTIONARY[k] = v[:-1]
    
    EVENT_INDEX = EVENT_INDEX - 1
    
    LABELED_TIMESTAMP = (None, None)
    
    # Now display the last event
    plt.clf()
    clear_output()
    print("The current Event Index is " + str(EVENT_INDEX))
    
    # Depending on the Mode, either use the product or maintenance events
    if "product" in EVENT_LABELING_MODE:
        
        if EVENT_INDEX > len(product_events)-1:
            print("THIS WAS THE LAST EVENT! YOU ARE DONE!")
            return 
        product_row = product_events.iloc[EVENT_INDEX]
        
     
        FIG, AX = plot_event_window(event_timestamp = product_row.Timestamp, 
            event_type = product_row.Product,
            window_size = WINDOW_SIZE,
            current_CREAM_day = current_CREAM_day,
            concurrent_events_dict = {
              "maintenance_events" : maintenance_events,
              "component_events" : component_events
            })
        
    elif "maintenance" in EVENT_LABELING_MODE:
        if EVENT_INDEX > len(maintenance_events)-1:
            print("THIS WAS THE LAST EVENT! YOU ARE DONE!")
            return 
        
        maintenance_row = maintenance_events.iloc[EVENT_INDEX]
        
        FIG, AX = plot_event_window(event_timestamp = maintenance_row.Timestamp, 
            event_type = maintenance_row.Activity,
            window_size = WINDOW_SIZE,
            current_CREAM_day = current_CREAM_day,
            concurrent_events_dict = {
              "product_events" : product_events,
              "component_events" : component_events
            })
        
    else:
        raise ValueError("This event labeling mode is not available. Please select maintenance or product!")
        
    FIG.show()
    display(button_box)
    
    return EVENT_DICTIONARY

# Only touch this area in the notebook to alter variables, like, for example, the day of interest and the path to the day

<div class="alert alert-danger">
    <h3>//ToDo</h3>
    <p>Please specify whether you label maintenance our product events </p>
</div>

In [ ]:
EVENT_LABELING_MODE = "maintenance" #"product" or "maintenance"

<div class="alert alert-danger">
    <h3>//ToDo</h3>
    <p>Please specify the path to the main-folder of "CREAM". </p>
</div>

In [ ]:
PATH_TO_DATA = os.path.abspath(os.path.join("..", "..", "Datasets", "CREAM", "CREAM""))

<div class="alert alert-danger">
    <h3>//ToDo</h3>
    <p>Please specify the path to location where you want to store the labels. </p>
</div>

In [ ]:
LABEL_DESTINATION_PATH = os.path.abspath(os.path.join("..", "..", "Datasets", "CREAM", "Labels"))

<div class="alert alert-danger">
    <h3>//ToDo</h3>
    <p>After you are done with a day, select the new day by altering the <b> CURRENT_DAY </b> variable and reset the notebook </p>
</div>

In [ ]:
""" 
The following days compromise the dataset: just copy&paste the day of interest to the CURRENT_DAY variable
"2018-08-23" , "2018-08-24" , 2018-08-25",  "2018-08-26" , "2018-08-27" , "2018-08-28" ,
"2018-08-29", "2018-08-30", "2018-08-31", "2018-09-01", "2018-09-02" , "2018-09-03" ,  "2018-09-04" ,
"2018-09-05", "2018-09-06", "2018-09-07", "2018-09-08" , "2018-09-09" , "2018-09-10", "2018-09-11", "2018-09-12" 
"2018-09-13" ,"2018-09-14" ,"2018-09-15" ,  "2018-09-16", "2018-09-17", "2018-09-18","2018-09-19"  , "2018-09-20" ,
"2018-09-21" , "2018-09-22" ,  "2018-09-23" ,"2018-09-24" ,"2018-09-25" ,"2018-09-26" , "2018-09-27", "2018-09-28" ,
"2018-09-29" , "2018-09-30" , "2018-10-01" ,"2018-10-02" , "2018-10-03" ,"2018-10-04", "2018-10-05" , "2018-10-06" ,
"2018-10-07", "2018-10-08" 

"""
CURRENT_DAY = "2018-10-08" 

# Here you can label the product events of this day

## Execute this cell to load the event files

<p> In case you want to use already refined event files, for example, in case you have already labeled the product events when
you start labeling the maintenance events, you can alter the paths below by uncommenting and commentin the respective lines. </p>

In [ ]:
#necessary for the plotting
# Load the events and already filter for the day of interest
# Load the maintenance and product events
day_path = os.path.join(PATH_TO_DATA, CURRENT_DAY)
current_CREAM_day = CREAM_Day(cream_day_location=day_path,use_buffer=True, buffer_size_files=2) 

maintenance_events = current_CREAM_day.load_machine_events(os.path.join(PATH_TO_DATA, "raw_coffee_maker_logs", "raw_maintenance_events.csv"), filter_day=True, raw_file=True)
#maintenance_events = current_CREAM_day.load_machine_events(os.path.join(PATH_TO_DATA, "maintenance_events.csv"), filter_day=True, raw_file=False)

product_events = current_CREAM_day.load_machine_events(os.path.join(PATH_TO_DATA, "raw_coffee_maker_logs","raw_product_events.csv"), filter_day=True, raw_file=True)
#product_events = current_CREAM_day.load_machine_events(os.path.join(PATH_TO_DATA, "product_events.csv"), filter_day=True, raw_file=False)

# Load the electrical component events
component_events = current_CREAM_day.load_component_events(os.path.join(PATH_TO_DATA, "component_events.csv"), filter_day=True, raw_file=False)

## Execute the cell below to get started with the labeling

<p> Place two clicks in the graph: One where the event starts, and one where it ends. They will appear in yellow </p>
<p> To ease labeling and to raise awareness for concurrent events the follwoing lines are displayed: </p>
    <p> Already labeled events (the start indices) are shown in yellow in the graph </p>
    <p> Appliance event labels are shown in dashed orange lines </p>
    <p> Any other product or maintenance event is show with a dashed red line </p>
<p> <b> The red line marks the point by that the event has to be finished latest! </b> </p>
<p> The short black lines represent one minute steps </p>
<p> If you want to delete the last click: Use the <b> "remove last click" </b> button (the middle, red one) </p>
<p> If you think you are done with this event, click the green <b> "next" </b> button to load the next event and save the preivous one </p>
<p> If you have selected <b> "next" </b> accidentially or still to remove the event borders you have labeled from the previous event,
select the other red button, the <b> "delete last entry" </b> one. </p>

<div class="alert alert-info">
 <h4>Empty Figure or not in interactive mode</h4>
    <p>If the plot does not load or is not in the interactive mode, reexecute the cell or reexcute the import cell</p>
</div>

<div class="alert alert-danger">
    <h3> Do not use the zoom and other capabilities from the plot toolbar</h3>
    <p>Clicks when zooming etc. also get registred as clicks for labels!</p>
</div>

In [ ]:
# Create and register Buttons
next_button =  Button(description="Next -> ",style=ButtonStyle(button_color='green'))
delete_button = Button(description=" <- Delete last entry",style=ButtonStyle(button_color='red'))
remove_last_click_button = Button(description="Remove last click",style=ButtonStyle(button_color='red'))
button_box = HBox([next_button, remove_last_click_button, delete_button])
next_button.on_click(on_next_clicked)
delete_button.on_click(on_delete_clicked)
remove_last_click_button.on_click(on_remove_last_click)
# Display first event --> event_index is set to zero for the start
# In case of erros or interruptions, provide another event index to the display_initial_event function
display_initial_event(event_index_p=0)

# Merge the single day .csv files into a single one

<p> After you are done with labeling all the days, you can execute this cell to merge the labels from the individual csv files into a single one! </p>

In [ ]:
def merge_label_files():
    """
    Function to merge the labeled files into a single final one
    """
    label_files = glob.glob(os.path.join(LABEL_DESTINATION_PATH, "*.csv"))
    label_files = [f for f in label_files 
                   if "previous" not in f 
                   and "labeled" not in f 
                   and EVENT_LABELING_MODE in f
                   and os.path.basename(f) != EVENT_LABELING_MODE+"_events.csv"]
    
    label_df_list = []
    for label_file in label_files:
            label_df_list.append(current_CREAM_day.load_machine_events(file_path=label_file, raw_file=False))
    
    overall_label_df = pd.concat(label_df_list, ignore_index=True, sort=False)
    overall_label_df.sort_values("Start_Timestamp", inplace=True)
   
    overall_label_df.reset_index(inplace=True, drop=True)
    overall_label_df.to_csv(os.path.join(LABEL_DESTINATION_PATH, EVENT_LABELING_MODE + "_events.csv"))

In [ ]:
merge_label_files()